In [1]:
%pip install --quiet mlflow plotly-express

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X, y = data.data, data.target
seed = 10 #Specify a seed value.

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = seed, stratify = y)


In [5]:
import mlflow

In [3]:
import os


tracking_uri = os.environ.get("MLFLOW_TRACKING_URI", "<fill in right before workshop>")
tracking_uri


'http://20.123.252.224:5000'

In [6]:

mlflow.set_tracking_uri(tracking_uri)

In [7]:
from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run():

  mlflow.log_param("name", "<fill in your name here>")

  #Logging the seed value to passed to the train_test_split function.
  mlflow.log_param("seed", seed)

  estimators = int(input("Estimator(s): "))

  #Model definition.
  rclf = RandomForestClassifier(n_estimators = estimators)

  mlflow.sklearn.autolog()
  rclf.fit(X_train, y_train)
  rclf.score(X_test, y_test)


2023/06/21 13:43:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/dunnkers/.pyenv/versions/3.10.9/envs/data-engineering-mlops-workshop-3-10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [8]:
mlflow.end_run()